### AWS Account Info

In [1]:
import sagemaker
sess = sagemaker.Session()

role = 'arn:aws:iam::062336837736:role/Developer'
account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name
print(region)
print(account)
print(role)
print(sess.boto_session.profile_name)

ca-central-1
062336837736
arn:aws:iam::062336837736:role/Developer
dev


### Build & Push Docker Image

#### Variables for Docker Image

In [2]:
image = 'cog_verse'
bucket_name   = sess.default_bucket()
base_job_name = 'cog-verse-training'
%env image {image}
%env account {account}
%env region {region}
%env bucket_name {bucket_name}
%env base_job_name = {base_job_name}

env: image=cog_verse
env: account=062336837736
env: region=ca-central-1
env: bucket_name=sagemaker-ca-central-1-062336837736
env: base_job_name=cog-verse-training


#### Build Image

In [4]:
%%sh 
bash ./build_and_push_docker.sh $image

Login Succeeded



Logging in with your password grants your terminal complete access to your account. 
For better security, log in with a limited-privilege personal access token. Learn more at https://docs.docker.com/go/access-tokens/
#1 [internal] load build definition from Dockerfile
#1 sha256:57772644dcdc5e72840d857dc48b19a69961ecf54c7375d2f55da348fad13690
#1 transferring dockerfile: 950B done
#1 DONE 0.0s

#2 [internal] load .dockerignore
#2 sha256:04f816664de95e51407cbead0b5f5d55fe0d302b5859d6a165c49ed5fb62ef2b
#2 transferring context: 135B done
#2 DONE 0.0s

#3 [internal] load metadata for docker.io/library/python:3.9
#3 sha256:5ceb849adf4ca2eed629b09d8add870d381c45f6f40f947be68f6595c97a8af8
#3 ...

#4 [auth] library/python:pull token for registry-1.docker.io
#4 sha256:39ee80147779d51de95bd27b3855509ec7a402cb109a76b935de3867ba4689f6
#4 DONE 0.0s

#3 [internal] load metadata for docker.io/library/python:3.9
#3 sha256:5ceb849adf4ca2eed629b09d8add870d381c45f6f40f947be68f6595c97a8af8
#3 DONE 1.2s

#5

#### Push Image to ECR

In [5]:
!docker push $account.dkr.ecr.$region.amazonaws.com/${image}:latest

The push refers to repository [062336837736.dkr.ecr.ca-central-1.amazonaws.com/cog_verse]

468a8530: Preparing 
955bb3f6: Preparing 
f4ed6dcd: Preparing 
bf18a086: Preparing 
a56e1308: Preparing 
e6d1e924: Preparing 
88911a5c: Preparing 
89fa82be: Preparing 
3ce83836: Preparing 
7063fb95: Preparing 
177efc0c: Preparing 
b0d193bc: Preparing 
5e673844: Preparing 
811d364e: Preparing 
260e173a: Preparing 
bb98b0dc: Preparing 
8387f467: Preparing 
4ed6dcd: Pushing  369.3MB/3.697GB4APushing  45.06MBPushing  55.08MB/3.697GBPushing  308.5MB/3.697GB

### Training

#### Local Test

In [ ]:
# Training setup
output_path = f"s3://{bucket_name}/{image}/output"
image_name = f"{account}.dkr.ecr.{region}.amazonaws.com/{image}:latest"

estimator = sagemaker.estimator.Estimator(image_uri=image_name,
                       base_job_name=base_job_name,
                       role=role, 
                       instance_count=1, 
                       output_path=output_path,
                       instance_type='local')
hyperparameters = {
    'main_args': "+experiment=simple_a2c/cartpole"
}
estimator.fit(hyperparameters=hyperparameters)

# Verification
print(f"output_path: {output_path}")
print(f"image_name: {image_name}")

#### AWS Run

In [ ]:
# Training setup
output_path = f"s3://{bucket_name}/{image}/output"
image_name = f"{account}.dkr.ecr.{region}.amazonaws.com/{image}:latest"
tag_name = [{'Key': 'cog-verse', 'Value': 'cog-verse-training'}]

estimator = sagemaker.estimator.Estimator(image_uri=image_name,
                       base_job_name=base_job_name,
                       role=role, 
                       instance_count=1, 
                       instance_type='ml.m5.large',
                       volume_size=16,
                       tags=tag_name,
                       source_dir='.cogment_verse',
                       output_path=output_path,
                       sagemaker_session=sess)
hyperparameters = {
    'main_args': "+experiment=simple_a2c/cartpole"
}
estimator.fit(hyperparameters=hyperparameters)

# Verification
print(f"output_path: {output_path}")
print(f"image_name: {image_name}")
